# Flight Dataset Clean

### Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Import Library

In [2]:
!pip install airportsdata
!pip install FlightRadarAPI
!pip install geopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 26.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import calendar
import airportsdata
import requests
from FlightRadar24.api import FlightRadar24API
pd.options.mode.chained_assignment = None  # default='warn'

### Read Flight Dataset (Original)

In [4]:
flight = pd.read_excel('/content/drive/My Drive/FYP2/Data_FlightRaw.xlsx')
print(flight.shape)
display(flight.head(2))

(60550, 17)


,airline_iata,airline_icao,flight_iata,flight_icao,flight_number,dep_iata,dep_time,dep_time_utc,arr_iata,arr_time,arr_time_utc,cs_airline_iata,cs_flight_number,cs_flight_iata,status,duration,delayed
0,MH,MAS,MH194,MAS194,194,KUL,2023-01-06 19:55:00,2023-01-06 11:55:00,BOM,2023-01-06 22:35:00,2023-01-06 17:05:00,NaN,NaN,NaN,scheduled,310.0,NaN
1,AK,AXM,AK6440,AXM6440,6440,KUL,2023-01-06 19:50:00,2023-01-06 11:50:00,KBR,2023-01-06 21:00:00,2023-01-06 13:00:00,NaN,NaN,NaN,scheduled,70.0,NaN


### Drop Columns
##### airline_iata, flight_icao, flight number, cs_airline_iata, cs_flight_number, cs_flight_iata,schedule

In [5]:
flightclean = flight[['flight_iata','airline_icao','dep_iata','dep_time','dep_time_utc','arr_iata','arr_time','arr_time_utc','duration','delayed']]
print(flightclean.shape)
display(flightclean.head(2))

(60550, 10)


,flight_iata,airline_icao,dep_iata,dep_time,dep_time_utc,arr_iata,arr_time,arr_time_utc,duration,delayed
0,MH194,MAS,KUL,2023-01-06 19:55:00,2023-01-06 11:55:00,BOM,2023-01-06 22:35:00,2023-01-06 17:05:00,310.0,NaN
1,AK6440,AXM,KUL,2023-01-06 19:50:00,2023-01-06 11:50:00,KBR,2023-01-06 21:00:00,2023-01-06 13:00:00,70.0,NaN


### Drop NA

In [6]:
flightclean = flightclean.dropna(subset=['dep_time_utc', 'arr_time_utc'])

### Clean Time
- Round time (10:45:00 -> 11:00:00)
- Extract Hour (11:00:00 -> 11)
- Extract Day (6/3/2023 -> Monday)

In [7]:
def roundHour(time):
    time = pd.to_datetime(time, format='%H:%M:%S')

    if time.minute >= 30:
        time = str((time.hour+1))+":00:00"
    else:
        time = str((time.hour))+":00:00"

    if time == "24:00:00":
        time = "00:00:00"

    time = pd.to_datetime(time,format='%H:%M:%S')

    return time.strftime('%H:%M:%S')

def getHour(time):
    time = pd.to_datetime(time,format='%H:%M:%S')

    if time == 24:
        return 0
    else:
        return time.hour

def getDay(date):
    date = pd.to_datetime(date,format='%d-%m-%Y')
    return calendar.day_name[date.weekday()]

def getWeekend(day):
  if (day == "Saturday") or (day == "Sunday"):
    return True
  else:
    return False

def getPartsofDay(hour):
  if(hour >= 5) and (hour < 12):
    return "Morning"
  elif (hour >= 12) and (hour < 17):
    return "Afternoon"
  elif (hour >= 17) and (hour < 21):
    return "Evening"
  else:
    return "Night"

In [8]:
#Format Time
flightclean['dep_Time_utc'] = pd.to_datetime(flightclean.dep_time_utc,format='%Y-%m-%d %H:%M:%S')
flightclean['dep_Time_local'] = pd.to_datetime(flightclean.dep_time,format='%Y-%m-%d %H:%M:%S')

flightclean = flightclean.drop('dep_time_utc', axis=1)
flightclean = flightclean.drop('dep_time', axis=1)

#Date
flightclean['dep_Date_utc'] = flightclean['dep_Time_utc'].dt.strftime('%d-%m-%Y')
flightclean['dep_Date_local'] = flightclean['dep_Time_local'].dt.strftime('%d-%m-%Y')

#Time Format
flightclean['dep_Time_utc'] = flightclean['dep_Time_utc'].dt.strftime('%H:%M:%S')
flightclean['dep_Time_local'] = flightclean['dep_Time_local'].dt.strftime('%H:%M:%S')

#Round the Time Column Local Time
# 8:10:00 -> 8:00:00
flightclean['dep_TimeRound_utc'] = flightclean['dep_Time_utc'].apply(roundHour)
flightclean['dep_TimeRound_local'] = flightclean['dep_Time_local'].apply(roundHour)

#Extract Hour and Day
flightclean['dep_Hour_utc'] = flightclean['dep_TimeRound_utc'].apply(getHour)
flightclean['dep_Day_utc'] = flightclean['dep_Date_utc'].apply(getDay)

flightclean['dep_Hour_local'] = flightclean['dep_TimeRound_local'].apply(getHour)
flightclean['dep_Day_local'] = flightclean['dep_Date_local'].apply(getDay)

flightclean['dep_PartofHour_local'] = flightclean['dep_Hour_local'].apply(getPartsofDay)
flightclean['dep_Weekend_local'] = flightclean['dep_Day_local'].apply(getWeekend)

flightclean = flightclean.drop('dep_TimeRound_utc', axis=1)
flightclean = flightclean.drop('dep_TimeRound_local', axis=1)
flightclean = flightclean.drop('dep_Time_local', axis=1)
flightclean = flightclean.drop('dep_Date_local', axis=1)


#Arrange Column
flightclean.insert(3,'dep_PartofHour_local', flightclean.pop('dep_PartofHour_local'))
flightclean.insert(3,'dep_Weekend_local', flightclean.pop('dep_Weekend_local'))

flightclean.insert(3,'dep_Hour_local', flightclean.pop('dep_Hour_local'))
flightclean.insert(3,'dep_Day_local', flightclean.pop('dep_Day_local'))

flightclean.insert(3,'dep_Hour_utc', flightclean.pop('dep_Hour_utc'))
flightclean.insert(3,'dep_Day_utc', flightclean.pop('dep_Day_utc'))

flightclean.insert(3,'dep_Time_utc', flightclean.pop('dep_Time_utc'))
flightclean.insert(3,'dep_Date_utc', flightclean.pop('dep_Date_utc'))

In [9]:
#Format Time
flightclean['arr_Time_utc'] = pd.to_datetime(flightclean.arr_time_utc,format='%Y-%m-%d %H:%M:%S')
flightclean['arr_Time_local'] = pd.to_datetime(flightclean.arr_time,format='%Y-%m-%d %H:%M:%S')

flightclean = flightclean.drop('arr_time_utc', axis=1)
flightclean = flightclean.drop('arr_time', axis=1)

#Date
flightclean['arr_Date_utc'] = flightclean['arr_Time_utc'].dt.strftime('%d-%m-%Y')
flightclean['arr_Date_local'] = flightclean['arr_Time_local'].dt.strftime('%d-%m-%Y')

#Time
flightclean['arr_Time_utc'] = flightclean['arr_Time_utc'].dt.strftime('%H:%M:%S')
flightclean['arr_Time_local'] = flightclean['arr_Time_local'].dt.strftime('%H:%M:%S')

#Round the Time Column
flightclean['arr_TimeRound_utc'] = flightclean['arr_Time_utc'].apply(roundHour)
flightclean['arr_TimeRound_local'] = flightclean['arr_Time_local'].apply(roundHour)

#Extract Hour and Day
flightclean['arr_Hour_utc'] = flightclean['arr_TimeRound_utc'].apply(getHour)
flightclean['arr_Day_utc'] = flightclean['arr_Date_utc'].apply(getDay)

flightclean['arr_Hour_local'] = flightclean['arr_TimeRound_local'].apply(getHour)
flightclean['arr_Day_local'] = flightclean['arr_Date_local'].apply(getDay)

flightclean['arr_PartofHour_local'] = flightclean['arr_Hour_local'].apply(getPartsofDay)
flightclean['arr_Weekend_local'] = flightclean['arr_Day_local'].apply(getWeekend)

flightclean = flightclean.drop('arr_TimeRound_utc', axis=1)
flightclean = flightclean.drop('arr_TimeRound_local', axis=1)
flightclean = flightclean.drop('arr_Time_local', axis=1)
flightclean = flightclean.drop('arr_Date_local', axis=1)

#Arrange Column
flightclean.insert(12,'arr_PartofHour_local', flightclean.pop('arr_PartofHour_local'))
flightclean.insert(12,'arr_Weekend_local', flightclean.pop('arr_Weekend_local'))

flightclean.insert(12,'arr_Hour_local', flightclean.pop('arr_Hour_local'))
flightclean.insert(12,'arr_Day_local', flightclean.pop('arr_Day_local'))

flightclean.insert(12,'arr_Hour_utc', flightclean.pop('arr_Hour_utc'))
flightclean.insert(12,'arr_Day_utc', flightclean.pop('arr_Day_utc'))

flightclean.insert(12,'arr_Time_utc', flightclean.pop('arr_Time_utc'))
flightclean.insert(12,'arr_Date_utc', flightclean.pop('arr_Date_utc'))

### Clean Delay

In [10]:
def delay(time):
    if time == 0:
        return False
    else:
        return True

flightclean['delayed'] = flightclean['delayed'].fillna(0)
flightclean['delayStatus'] = flightclean['delayed'].apply(delay)

### Airport Info

In [11]:
airports = airportsdata.load('IATA')

arrAirportName= []
arrAirportCity= []
arrAirportLat= []
arrAirportLon= []
depAirportName= []
depAirportCity= []
depAirportLat= []
depAirportLon= []

for index, row in flightclean.iterrows():
    arrAirportName.append(airports[row['arr_iata']]['name'])
    arrAirportCity.append(airports[row['arr_iata']]['city'])
    arrAirportLat.append(airports[row['arr_iata']]['lat'])
    arrAirportLon.append(airports[row['arr_iata']]['lon'])

    depAirportName.append(airports[row['dep_iata']]['name'])
    depAirportCity.append(airports[row['dep_iata']]['city'])
    depAirportLat.append(airports[row['dep_iata']]['lat'])
    depAirportLon.append(airports[row['dep_iata']]['lon'])


flightclean['dep_Airport'] = depAirportName
flightclean['dep_City'] = depAirportCity
flightclean['dep_Lat'] = depAirportLat
flightclean['dep_Lon'] = depAirportLon

flightclean['arr_Airport'] = arrAirportName
flightclean['arr_City'] = arrAirportCity
flightclean['arr_Lat'] = arrAirportLat
flightclean['arr_Lon'] = arrAirportLon

flightclean.insert(3,'dep_Lon', flightclean.pop('dep_Lon'))
flightclean.insert(3,'dep_Lat', flightclean.pop('dep_Lat'))
flightclean.insert(3,'dep_Airport', flightclean.pop('dep_Airport'))
flightclean.insert(3,'dep_City', flightclean.pop('dep_City'))

flightclean.insert(16,'arr_Lon', flightclean.pop('arr_Lon'))
flightclean.insert(16,'arr_Lat', flightclean.pop('arr_Lat'))
flightclean.insert(16,'arr_Airport', flightclean.pop('arr_Airport'))
flightclean.insert(16,'arr_City', flightclean.pop('arr_City'))

In [12]:
from FlightRadar24.api import FlightRadar24API
fr_api = FlightRadar24API()
airline_info = fr_api.get_airlines()

icao_to_name = {}
for airline in airline_info:
    icao_to_name[airline["ICAO"]] = airline["Name"]

flightclean["airline_icao"] = flightclean["airline_icao"].map(icao_to_name)

flightclean = flightclean.rename(columns={"airline_icao": "Airline"})

### Map Flight Model into Flight

In [13]:
flightmodel = pd.read_csv('/content/drive/My Drive/FYP2/flightmodel.csv')
flightmodel = flightmodel.rename(columns={"flight_iata": "iata"})
flightmodel.head(3)

,iata,Aircraft_Model,Aircraft_isTurboProp,Aircraft_isWidebody
0,MH194,Airbus A330-300,False,True
1,AK6440,Airbus A320,False,False
2,MH196,Airbus A330-300,False,True


In [14]:
# use merge to map df1 data into df2 based on flight_iata
flightclean = pd.merge(flightclean, flightmodel, how="left", left_on=['flight_iata'], right_on=['iata']);

flightclean = flightclean.drop(['iata'],axis=1)

### Flight Type (Domestics/ International)

In [ ]:
from geopy.geocoders import Nominatim

merged = pd.concat([flightclean['arr_City'], flightclean['dep_City']]).drop_duplicates()

countrydf = pd.DataFrame()
countrydf['City'] = merged.tolist()

geolocator = Nominatim(user_agent="my_app")

def country(city):
  location = geolocator.geocode(city)

  if location is not None:
    return location.raw['display_name'].split(',')[-1].strip()
  else:
    return None

countrydf['Country'] = countrydf['City'].apply(country)
city_to_country = {}
for index, row in countrydf.iterrows():
  city_to_country[row['City']] = row['Country']

In [ ]:
flightclean['Dep_Country'] = flightclean['dep_City'].map(city_to_country)
flightclean['Arr_Country'] = flightclean['arr_City'].map(city_to_country)

flighttype = []

for index, row in flightclean.iterrows():
  if row['Dep_Country'] == row['Arr_Country']:
    flighttype.append('Domestic')
  else:
    flighttype.append('International')

flightclean['Flight_Type'] = flighttype

flightclean = flightclean.drop(['Dep_Country','Arr_Country'],axis=1)
flightclean

### Get CSV Files

In [ ]:
from google.colab import files
flightclean.to_excel('FYP2_FlightDataset_Final.xlsx', encoding = 'utf-8-sig',index=False)
files.download('FYP2_FlightDataset_Final.xlsx')

/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>